In [12]:
import pandas as pd
import re

In [54]:
# org_ntw = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/네트워크_merged.csv')
# org_md = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/미디어_merged.csv')
# org_site = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/사이트_merged.csv')
# org_kwd = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/키워드_merged.csv')

naver_kwd = pd.read_csv('/Users/young/Documents/git/Oct.25__adhoc_mkt.centum/data/kwd.csv',header=1)

In [15]:
ntw_columns = ['media','network','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
kwd_columns = ['media','keyword','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
site_columns = ['media','site','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
md_columns = ['media','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']

org_ntw.columns = ntw_columns
org_kwd.columns = kwd_columns
org_site.columns = site_columns
org_md.columns = md_columns


# 예시: churnRate, conv_rev에서 %와 ₩를 제거하고 int로 변환하도록 apply 함수 사용
for df in [org_ntw, org_kwd, org_site, org_md]:
    if 'churnRate' in df.columns:
        df['churnRate'] = df['churnRate'].apply(lambda x: int(str(x).replace('%', '')))
    if 'conv_rev' in df.columns:
        df['conv_rev'] = df['conv_rev'].apply(lambda x: int(str(x).replace('₩', '').replace(',', '')))
    df['date'] = pd.to_datetime(df['date'])

org_ntw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   media            162 non-null    object        
 1   network          162 non-null    object        
 2   totalClick       162 non-null    int64         
 3   validClick       162 non-null    int64         
 4   churnRate        162 non-null    int64         
 5   avgSesstionTime  162 non-null    object        
 6   conv_ask         162 non-null    int64         
 7   conv_join        162 non-null    int64         
 8   conv_order       162 non-null    int64         
 9   conv_rev         162 non-null    int64         
 10  date             162 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 14.1+ KB


,media,network,totalClick,validClick,churnRate,avgSesstionTime,conv_ask,conv_join,conv_order,conv_rev,date
0,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-05
1,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-05
2,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-06
3,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-06
4,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-08-18
...,...,...,...,...,...,...,...,...,...,...,...
157,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-02
158,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-03
159,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-03
160,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-04


In [99]:
# org_ntw.groupby(['media','network']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
# org_site.groupby(['media']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
# org_md.groupby(['media']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
agg_kwd = org_kwd.groupby(['media','keyword']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).reset_index()
agg_kwd['convs'] = agg_kwd['conv_ask'] + agg_kwd['conv_join'] + agg_kwd['conv_order'] + agg_kwd['conv_rev']

In [100]:
# naver_kwd = naver_kwd[['키워드','총비용(VAT포함,원)']]
naver_kwd.columns = ['keyword','cost'] 
naver_kwd = naver_kwd.query('keyword != "-"')
# 키워드별 중복값이 있을 때 카테시안 곱이 발생하지 않도록 naver_kwd의 'keyword'별로 비용을 집계한 뒤 머지
naver_kwd_single = naver_kwd.groupby('keyword', as_index=False).agg({'cost':'sum'})
agg_kwd = pd.merge(naver_kwd_single, agg_kwd, left_on='keyword', right_on='keyword', how='left')
agg_kwd.fillna(0, inplace=True)

In [ ]:
agg_kwd.sort_values(by='cost', ascending=False)

sum(agg_kwd['cost']) / sum(agg_kwd['convs'])
agg_kwd.loc[agg_kwd['convs'] == 0, 'cost'].sum() / agg_kwd['convs'].sum()
# sum(agg_kwd['cost'])
# sum(agg_kwd[agg_kwd['convs'] == 0].cost) / agg_kwd.cost.sum()
# sum(agg_kwd[agg_kwd['convs'] != 0].cost)

3782.0802526186076

In [105]:
agg_kwd['cost_per_conv'] = round(agg_kwd['cost'] / agg_kwd['convs'],2)
agg_kwd

,keyword,cost,media,totalClick,validClick,conv_ask,conv_join,conv_order,conv_rev,convs,cost_per_conv
0,TMJ,1144,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf
1,TMJ치과,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,강서구강내과전문의,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,강서구강내과치과,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,강서구구강내과,1760,네이버,243.0,243.0,0.0,0.0,0.0,0.0,0.0,inf
5,강서구구강내과전문의,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6,강서구구강내과치과,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7,강서구이갈이,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,강서구이갈이스플린트,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9,강서구코골이,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
org_site.

(Timestamp('2025-08-18 00:00:00'), Timestamp('2025-11-06 00:00:00'))

In [33]:
org_site.media.unique()

array(['네이버', '구글'], dtype=object)